In [1]:
from collections import defaultdict

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [42]:
fbs = pd.read_csv("data/FBS_data.csv", encoding='gbk')
gt = pd.read_csv("data/GT_data.csv", encoding='gbk')
qcl = pd.read_csv("data/QCL_data.csv", encoding='gbk')
weather = pd.read_csv("data/cleaned_yearly_weather_data.csv", encoding='gbk')

### noaa iso preprocessing

In [43]:
with open("data/country_codes.txt", 'r', encoding='utf-8') as f:
        country_codes = f.readlines()
fips_list = [i.strip().split('\t') for i in country_codes[1:]]
fips_dict = {i[1]: i[0] for i in fips_list}
fips_iso_dict = {i[1]: i[2] for i in fips_list}

In [45]:
weather = weather.dropna(subset=['NAME'])
weather['fips'] = [i.split(',')[-1].strip() for i in list(weather['NAME'])]
countries = list(weather['fips'])
country_names = []
iso = []
for country in countries:
    if fips_dict.get(country):
        country_names.append(fips_dict[country])
        iso.append(fips_iso_dict[country])
    else:
        country_part_dict = {k: v for k, v in fips_dict.items() if country in k or k in country}
        if len(country_part_dict) == 1:
            country_names.append(list(country_part_dict.values())[0])
            iso.append(fips_iso_dict[list(country_part_dict.keys())[0]])
        else:
            country_names.append(None)
            iso.append(None)

In [46]:
weather['country_name'] = country_names
weather['iso'] = iso
weather['DATE'] = weather['DATE'].astype('int')

In [61]:
weather_iso = weather

### fao iso preprocessing

In [51]:
fao_code = pd.read_csv("data/fao_country.csv")

In [55]:
fbs_iso = pd.merge(fbs, fao_code[["Country", "ISO2 Code"]],left_on="Area",right_on="Country", how="inner")

In [57]:
gt_iso = pd.merge(gt, fao_code[["Country", "ISO2 Code"]],left_on="Area",right_on="Country", how="inner")

In [59]:
qcl_iso = pd.merge(qcl, fao_code[["Country", "ISO2 Code"]],left_on="Area",right_on="Country", how="inner")

In [68]:
weather_mean = weather_iso[['TAVG', 'TMAX', 'TMIN', 'PRCP', 'DATE','iso']].groupby(by=['iso','DATE']).mean()

In [90]:
weather_mean = weather_mean.reset_index()

In [91]:
weather_mean

,iso,DATE,TAVG,TMAX,TMIN,PRCP
0,AE,2012,NaN,35.600000,NaN,NaN
1,AM,2004,13.000000,18.400000,7.600000,139.2
2,AM,2005,NaN,NaN,NaN,NaN
3,AM,2006,NaN,11.200000,NaN,NaN
4,AM,2007,NaN,9.950000,NaN,NaN
...,...,...,...,...,...,...
969,ZA,2010,15.840000,23.950000,8.100000,NaN
970,ZA,2011,16.700000,23.666667,9.120000,NaN
971,ZA,2012,13.733333,21.250000,6.950000,NaN
972,ZA,2013,13.733333,22.000000,6.700000,2098.4


In [75]:
fbs_iso_wheat = fbs_iso[fbs_iso['Item Code']==2511]

In [81]:
fbs_iso_piv = pd.pivot_table(fbs_iso,values='Value', columns=['Element'], aggfunc="mean", index=['Item','ISO2 Code', 'Year'])

In [88]:
fbs_apples = fbs_iso_piv.loc['Apples and products', :, :].reset_index()

In [92]:
apples_weather = pd.merge(weather_mean, fbs_apples, left_on=['iso', 'DATE'], right_on=['ISO2 Code', 'Year'], how='inner')

In [95]:
apples_weather

,iso,DATE,TAVG,TMAX,TMIN,PRCP,ISO2 Code,Year,Fat supply quantity (g),Food supply (kcal),Protein supply quantity (g)
0,AE,2012,NaN,35.600000,NaN,NaN,AE,2012,285.25,47428.34,101.38
1,AM,2010,NaN,12.950000,NaN,NaN,AM,2010,75.92,15186.20,59.86
2,AM,2012,NaN,11.100000,NaN,NaN,AM,2012,237.93,41469.45,120.84
3,AM,2013,NaN,10.900000,NaN,NaN,AM,2013,227.78,40626.72,120.39
4,AM,2014,NaN,11.800000,NaN,4739.3,AM,2014,209.13,36989.26,97.41
...,...,...,...,...,...,...,...,...,...,...,...
391,ZA,2010,15.840000,23.950000,8.100000,NaN,ZA,2010,474.00,99265.00,422.00
392,ZA,2011,16.700000,23.666667,9.120000,NaN,ZA,2011,616.00,127129.00,594.00
393,ZA,2012,13.733333,21.250000,6.950000,NaN,ZA,2012,403.00,96701.00,478.00
394,ZA,2013,13.733333,22.000000,6.700000,2098.4,ZA,2013,140.00,58290.00,244.00


In [98]:
apples_weather[['TMIN','Fat supply quantity (g)', 'Food supply (kcal)', 'Protein supply quantity (g)']].corr()

,TMIN,Fat supply quantity (g),Food supply (kcal),Protein supply quantity (g)
TMIN,1.000000,-0.135357,-0.112859,-0.062019
Fat supply quantity (g),-0.135357,1.000000,0.997551,0.981396
Food supply (kcal),-0.112859,0.997551,1.000000,0.989205
Protein supply quantity (g),-0.062019,0.981396,0.989205,1.000000


In [102]:
fbs_all = fbs_iso_piv.reset_index()[['ISO2 Code', 'Year', 'Fat supply quantity (g)', 'Food supply (kcal)', 'Protein supply quantity (g)']].groupby(by=['ISO2 Code','Year']).sum()

In [105]:
all_weather = pd.merge(weather_mean, fbs_all.reset_index(), left_on=['iso', 'DATE'], right_on=['ISO2 Code', 'Year'], how='inner')

In [109]:
all_weather[['TMAX','Fat supply quantity (g)', 'Food supply (kcal)', 'Protein supply quantity (g)']].corr()

,TMAX,Fat supply quantity (g),Food supply (kcal),Protein supply quantity (g)
TMAX,1.000000,0.026626,0.080540,0.054723
Fat supply quantity (g),0.026626,1.000000,0.979111,0.990120
Food supply (kcal),0.080540,0.979111,1.000000,0.994992
Protein supply quantity (g),0.054723,0.990120,0.994992,1.000000


In [110]:
fbs_all

Element         Fat supply quantity (g)  Food supply (kcal)  \
ISO2 Code Year                                                
AE        2010                344265.93          9246800.83   
          2011                350880.07          9726324.88   
          2012                369366.04         10052558.96   
          2013                362081.25         10264580.95   
          2014                363544.23         10260530.31   
...                                 ...                 ...   
ZW        2010                350401.55         10339015.79   
          2011                365246.72         10482455.46   
          2012                358534.23         10817298.91   
          2013                350416.59         10900687.10   
          2014                342126.40         10682186.90   

Element         Protein supply quantity (g)  
ISO2 Code Year                               
AE        2010                    220871.20  
          2011                    254196.03  
          2012                    252457.04  
          2013                    270503.05  
          2014                    266958.36  
...                                     ...  
ZW        2010                    299711.25  
          2011                    296686.15  
          2012                    305352.79  
          2013                    306736.20  
          2014                    290741.15  

[1044 rows x 3 columns]

In [112]:
all_weather_us = all_weather[all_weather['iso']=='US']

In [118]:
all_weather.corr()

/var/folders/8m/wb8xgy694zvddt58c2wzjz_80000gn/T/ipykernel_21082/315041841.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  all_weather.corr()


,DATE,TAVG,TMAX,TMIN,PRCP,Year,Fat supply quantity (g),Food supply (kcal),Protein supply quantity (g)
DATE,1.000000,0.008804,-0.040573,0.026295,0.041776,1.000000,0.012558,0.013279,0.011908
TAVG,0.008804,1.000000,0.970783,0.985082,-0.081418,0.008804,0.031165,0.107196,0.074931
TMAX,-0.040573,0.970783,1.000000,0.926876,-0.144534,-0.040573,0.026626,0.080540,0.054723
TMIN,0.026295,0.985082,0.926876,1.000000,0.148811,0.026295,-0.028694,0.051739,0.018674
PRCP,0.041776,-0.081418,-0.144534,0.148811,1.000000,0.041776,-0.078635,-0.073294,-0.070926
Year,1.000000,0.008804,-0.040573,0.026295,0.041776,1.000000,0.012558,0.013279,0.011908
Fat supply quantity (g),0.012558,0.031165,0.026626,-0.028694,-0.078635,0.012558,1.000000,0.979111,0.990120
Food supply (kcal),0.013279,0.107196,0.080540,0.051739,-0.073294,0.013279,0.979111,1.000000,0.994992
Protein supply quantity (g),0.011908,0.074931,0.054723,0.018674,-0.070926,0.011908,0.990120,0.994992,1.000000


#### greenhouse from food

In [120]:
gt_iso_piv = pd.pivot_table(gt_iso,values='Value', columns=['Element'], aggfunc="mean", index=['Item','ISO2 Code', 'Year'])

In [122]:
all_gt_weather = pd.merge(weather_mean, gt_iso_piv.reset_index().groupby(by=['ISO2 Code', 'Year']).sum(), left_on=['iso', 'DATE'], right_on=['ISO2 Code', 'Year'], how='inner')

/var/folders/8m/wb8xgy694zvddt58c2wzjz_80000gn/T/ipykernel_21082/1358074706.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  all_gt_weather = pd.merge(weather_mean, gt_iso_piv.reset_index().groupby(by=['ISO2 Code', 'Year']).sum(), left_on=['iso', 'DATE'], right_on=['ISO2 Code', 'Year'], how='inner')


In [124]:
all_gt_weather.corr()

/var/folders/8m/wb8xgy694zvddt58c2wzjz_80000gn/T/ipykernel_21082/1171083784.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  all_gt_weather.corr()


,DATE,TAVG,TMAX,TMIN,PRCP,Emissions (CO2),Emissions (CO2eq) from CH4 (AR5),Emissions (CO2eq) from F-gases (AR5)
DATE,1.000000,0.100582,0.047318,0.129446,0.076204,-0.007317,-0.019678,-0.001828
TAVG,0.100582,1.000000,0.979088,0.985149,0.294506,0.307770,0.182596,-0.085054
TMAX,0.047318,0.979088,1.000000,0.938547,0.190784,0.305596,0.184994,-0.065972
TMIN,0.129446,0.985149,0.938547,1.000000,0.394200,0.306162,0.121468,-0.126429
PRCP,0.076204,0.294506,0.190784,0.394200,1.000000,0.026317,-0.123225,-0.030641
Emissions (CO2),-0.007317,0.307770,0.305596,0.306162,0.026317,1.000000,0.420943,-0.097265
Emissions (CO2eq) from CH4 (AR5),-0.019678,0.182596,0.184994,0.121468,-0.123225,0.420943,1.000000,0.278638
Emissions (CO2eq) from F-gases (AR5),-0.001828,-0.085054,-0.065972,-0.126429,-0.030641,-0.097265,0.278638,1.000000


#### yield

In [125]:
qcl_iso

,Domain Code,Domain,Area Code (FAO),Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Country,ISO2 Code
0,QCL,Crops and livestock products,2,Afghanistan,5419,Yield,221,"Almonds, in shell",2004,2004,hg/ha,12250,Afghanistan,AF
1,QCL,Crops and livestock products,2,Afghanistan,5419,Yield,221,"Almonds, in shell",2005,2005,hg/ha,13281,Afghanistan,AF
2,QCL,Crops and livestock products,2,Afghanistan,5419,Yield,221,"Almonds, in shell",2006,2006,hg/ha,16667,Afghanistan,AF
3,QCL,Crops and livestock products,2,Afghanistan,5419,Yield,221,"Almonds, in shell",2007,2007,hg/ha,26234,Afghanistan,AF
4,QCL,Crops and livestock products,2,Afghanistan,5419,Yield,221,"Almonds, in shell",2008,2008,hg/ha,35000,Afghanistan,AF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173929,QCL,Crops and livestock products,5817,Net Food Importing Developing Countries,5419,Yield,135,Yautia,2010,2010,hg/ha,77467,Net Food Importing Developing Countries,F5817
173930,QCL,Crops and livestock products,5817,Net Food Importing Developing Countries,5419,Yield,135,Yautia,2011,2011,hg/ha,85847,Net Food Importing Developing Countries,F5817
173931,QCL,Crops and livestock products,5817,Net Food Importing Developing Countries,5419,Yield,135,Yautia,2012,2012,hg/ha,92246,Net Food Importing Developing Countries,F5817
173932,QCL,Crops and livestock products,5817,Net Food Importing Developing Countries,5419,Yield,135,Yautia,2013,2013,hg/ha,100493,Net Food Importing Developing Countries,F5817


In [126]:
qcl_iso_piv = pd.pivot_table(qcl_iso,values='Value', columns=['Element'], aggfunc="mean", index=['Item','ISO2 Code', 'Year'])

In [132]:
all_qcl_weather = pd.merge(weather_mean, qcl_iso_piv.reset_index().groupby(by=['ISO2 Code', 'Year']).sum(), left_on=['iso', 'DATE'], right_on=['ISO2 Code', 'Year'], how='inner')

/var/folders/8m/wb8xgy694zvddt58c2wzjz_80000gn/T/ipykernel_21082/2920248973.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  all_qcl_weather = pd.merge(weather_mean, qcl_iso_piv.reset_index().groupby(by=['ISO2 Code', 'Year']).sum(), left_on=['iso', 'DATE'], right_on=['ISO2 Code', 'Year'], how='inner')


In [135]:
all_qcl_weather.corr()

/var/folders/8m/wb8xgy694zvddt58c2wzjz_80000gn/T/ipykernel_21082/1883258349.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  all_qcl_weather.corr()


,DATE,TAVG,TMAX,TMIN,PRCP,Yield,Yield/Carcass Weight
DATE,1.000000,0.100245,0.053901,0.123373,0.046300,0.052843,-0.030439
TAVG,0.100245,1.000000,0.975805,0.983475,0.247652,-0.053995,-0.249778
TMAX,0.053901,0.975805,1.000000,0.930556,0.131680,-0.044346,-0.245181
TMIN,0.123373,0.983475,0.930556,1.000000,0.344904,-0.122821,-0.308909
PRCP,0.046300,0.247652,0.131680,0.344904,1.000000,-0.345230,-0.373254
Yield,0.052843,-0.053995,-0.044346,-0.122821,-0.345230,1.000000,0.507592
Yield/Carcass Weight,-0.030439,-0.249778,-0.245181,-0.308909,-0.373254,0.507592,1.000000


In [153]:
qcl_iso_apple_piv = pd.pivot_table(qcl_iso[qcl_iso['Item']=='Bananas'],values='Value', columns=['Element'], aggfunc="mean", index=['Item','ISO2 Code', 'Year'])

In [154]:
all_apple_weather = pd.merge(weather_mean, qcl_iso_apple_piv.reset_index().groupby(by=['ISO2 Code', 'Year']).sum(), left_on=['iso', 'DATE'], right_on=['ISO2 Code', 'Year'], how='inner')

/var/folders/8m/wb8xgy694zvddt58c2wzjz_80000gn/T/ipykernel_21082/272695244.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  all_apple_weather = pd.merge(weather_mean, qcl_iso_apple_piv.reset_index().groupby(by=['ISO2 Code', 'Year']).sum(), left_on=['iso', 'DATE'], right_on=['ISO2 Code', 'Year'], how='inner')


In [155]:
all_apple_weather.corr()

/var/folders/8m/wb8xgy694zvddt58c2wzjz_80000gn/T/ipykernel_21082/2603097876.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  all_apple_weather.corr()


,DATE,TAVG,TMAX,TMIN,PRCP,Yield
DATE,1.000000,0.108913,0.046516,0.137247,0.004535,0.037805
TAVG,0.108913,1.000000,0.945629,0.971185,0.458248,-0.210675
TMAX,0.046516,0.945629,1.000000,0.864469,0.237454,-0.036684
TMIN,0.137247,0.971185,0.864469,1.000000,0.547399,-0.272584
PRCP,0.004535,0.458248,0.237454,0.547399,1.000000,-0.483855
Yield,0.037805,-0.210675,-0.036684,-0.272584,-0.483855,1.000000


In [159]:
all_apple_weather[['PRCP','Yield']].dropna()

,PRCP,Yield
1,1086.100000,208096
11,668.008913,236845
12,640.172571,256317
13,543.754269,167787
14,693.366422,182810
...,...,...
330,803.296707,181845
331,872.684593,181923
332,898.112835,187701
335,1804.600000,496475


In [14]:
str_f = '''=IF(AND(M2>=95, M2<=100), "A+", IF(AND(M2>=90, M2<95), "A", IF(AND(M2>=85, M2<90), "A-", IF(AND(M2>=80, M2<85), "B+", IF(AND(M2>=75, M2<80), "B", IF(AND(M2>=70, M2<75), "B-", IF(AND(M2>=65, M2<70), "C+", IF(AND(M2>=60, M2<65), "C", IF(AND(M2>=55, M2<60), "C-", IF(AND(M2>=50, M2<55), "D+", IF(AND(M2>=45, M2<50), "D", IF(AND(M2>=40, M2<45), "D-", IF(AND(M2>=35, M2<40), "E+", IF(AND(M2>=30, M2<35), "E", IF(AND(M2>=25, M2<30), "E-", IF(AND(M2>=20, M2<25), "F+", IF(AND(M2>=15, M2<20), "F", IF(AND(M2>=10, M2<15), "F-", IF(AND(M2>=8, M2<10), "G+", IF(AND(M2>=5, M2<8), "G", IF(AND(M2>=2, M2<5), "G-", "NG")))))))))))))))))))))
'''
stra = str_f.replace("M2", "K2")

print(stra)


=IF(AND(K2>=95, K2<=100), "A+", IF(AND(K2>=90, K2<95), "A", IF(AND(K2>=85, K2<90), "A-", IF(AND(K2>=80, K2<85), "B+", IF(AND(K2>=75, K2<80), "B", IF(AND(K2>=70, K2<75), "B-", IF(AND(K2>=65, K2<70), "C+", IF(AND(K2>=60, K2<65), "C", IF(AND(K2>=55, K2<60), "C-", IF(AND(K2>=50, K2<55), "D+", IF(AND(K2>=45, K2<50), "D", IF(AND(K2>=40, K2<45), "D-", IF(AND(K2>=35, K2<40), "E+", IF(AND(K2>=30, K2<35), "E", IF(AND(K2>=25, K2<30), "E-", IF(AND(K2>=20, K2<25), "F+", IF(AND(K2>=15, K2<20), "F", IF(AND(K2>=10, K2<15), "F-", IF(AND(K2>=8, K2<10), "G+", IF(AND(K2>=5, K2<8), "G", IF(AND(K2>=2, K2<5), "G-", "NG")))))))))))))))))))))



In [4]:
print(stra)

=IF(AND(H2>=95, H2<=100), "A+", IF(AND(H2>=90, H2<95), "A", IF(AND(H2>=85, H2<90), "A-", IF(AND(H2>=80, H2<85), "B+", IF(AND(H2>=75, H2<80), "B", IF(AND(H2>=70, H2<75), "B-", IF(AND(H2>=65, H2<70), "C+", IF(AND(H2>=60, H2<65), "C", IF(AND(H2>=55, H2<60), "C-", IF(AND(H2>=50, H2<55), "D+", IF(AND(H2>=45, H2<50), "D", IF(AND(H2>=40, H2<45), "D-", IF(AND(H2>=35, H2<40), "E+", IF(AND(H2>=30, H2<35), "E", IF(AND(H2>=25, H2<30), "E-", IF(AND(H2>=20, H2<25), "F+", IF(AND(H2>=15, H2<20), "F", IF(AND(H2>=10, H2<15), "F-", IF(AND(H2>=8, H2<10), "G+", IF(AND(H2>=5, H2<8), "G", IF(AND(H2>=2, H2<5), "G-", "NG")))))))))))))))))))))

